https://docs.cleanlab.ai/stable/tutorials/token_classification.html

In [18]:
class_names = ['생활문화', '스포츠', '정치', '사회', 'IT과학', '경제', '세계']

# 클래스와 인덱스 매핑
class_to_idx = {
    '생활문화': 0,
    '스포츠': 1,
    '정치': 2,
    '사회': 3,
    'IT과학': 4,
    '경제': 5,
    '세계': 6
}

In [19]:
import torch
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from cleanlab.filter import find_label_issues
from cleanlab import Datalab
from cleanlab.rank import get_label_quality_scores

In [34]:
train_df = pd.read_csv('../data/augmentation/simon_dataset_duplicate_noise_only_first.csv') #train 2800개
preliminary_labels = train_df['target']

output_df = pd.read_csv('../data/augmentation/simon_dataset_duplicate_noise_only_first_with_prob.csv')

In [35]:
model_pred_probs = output_df[[f'prob_{i}' for i in range(7)]].values

In [36]:
# lab = Datalab(data=output_df, label_name="target") #output_df를 분석, 'target'에 레이블이 있음
# lab.find_issues(pred_probs=model_pred_probs) #문제(이슈)찾기 함수

In [37]:
# lab.report()

In [38]:
# from cleanlab.dataset import health_summary
# health_stats = health_summary(
#     labels=train_df['target'],
#     pred_probs=model_pred_probs,
#     class_names=class_names
# )

In [39]:
label_issues = find_label_issues(
    labels=preliminary_labels,       # 초기 라벨
    pred_probs=model_pred_probs,     # 모델의 예측 확률
)

In [40]:
from cleanlab.dataset import health_summary
health_stats = health_summary(
    labels=preliminary_labels,
    pred_probs=model_pred_probs,
    class_names=class_names
)

----------------------------------------------------------
|  Generating a Cleanlab Dataset Health Summary          |
|   for your dataset with 6,348 examples and 7 classes.  |
|  Note, Cleanlab is not a medical doctor... yet.        |
----------------------------------------------------------

Overall Class Quality and Noise across your dataset (below)
------------------------------------------------------------ 



,Class Name,Class Index,Label Issues,Inverse Label Issues,Label Noise,Inverse Label Noise,Label Quality Score
0,생활문화,0,0,0,0.0,0.0,1.0
1,스포츠,1,0,0,0.0,0.0,1.0
2,정치,2,0,0,0.0,0.0,1.0
3,사회,3,0,0,0.0,0.0,1.0
4,IT과학,4,0,0,0.0,0.0,1.0
5,경제,5,0,0,0.0,0.0,1.0
6,세계,6,0,0,0.0,0.0,1.0



Class Overlap. In some cases, you may want to merge classes in the top rows (below)
-----------------------------------------------------------------------------------



,Class Name A,Class Name B,Class Index A,Class Index B,Num Overlapping Examples,Joint Probability
0,생활문화,스포츠,0,1,0,0.0
1,생활문화,정치,0,2,0,0.0
2,생활문화,사회,0,3,0,0.0
3,생활문화,IT과학,0,4,0,0.0
4,생활문화,경제,0,5,0,0.0
5,생활문화,세계,0,6,0,0.0
6,스포츠,정치,1,2,0,0.0
7,스포츠,사회,1,3,0,0.0
8,스포츠,IT과학,1,4,0,0.0
9,스포츠,경제,1,5,0,0.0



 * Overall, about 0% (0 of the 6,348) labels in your dataset have potential issues.
 ** The overall label health score for this dataset is: 1.00.

Generated with <3 from Cleanlab.



In [41]:
import numpy as np
# 라벨 오류 가능성이 있는 샘플의 인덱스만 추출
label_issue_indices = np.where(label_issues)[0]

# 오류 가능성이 높은 샘플의 인덱스만 리스트로 가져오기
issue_indices = train_df.index[label_issue_indices]
issue_indices

Index([], dtype='int64')

In [29]:
train_df = train_df.drop(columns=['Unnamed: 0'])

In [30]:
# 오류 샘플에서 모델이 예측한 라벨 보기
corrected_labels = [model_pred_probs[idx].argmax() for idx in label_issue_indices]

In [31]:
# 라벨을 모델의 예측 라벨로 교체
train_df.loc[label_issue_indices, 'target'] = corrected_labels

In [32]:
from cleanlab.dataset import health_summary
health_stats = health_summary(
    labels=train_df['target'],
    pred_probs=model_pred_probs,
    class_names=class_names
)

----------------------------------------------------------
|  Generating a Cleanlab Dataset Health Summary          |
|   for your dataset with 1,213 examples and 7 classes.  |
|  Note, Cleanlab is not a medical doctor... yet.        |
----------------------------------------------------------

Overall Class Quality and Noise across your dataset (below)
------------------------------------------------------------ 



,Class Name,Class Index,Label Issues,Inverse Label Issues,Label Noise,Inverse Label Noise,Label Quality Score
0,생활문화,0,0,0,0.0,0.0,1.0
1,스포츠,1,0,0,0.0,0.0,1.0
2,정치,2,0,0,0.0,0.0,1.0
3,사회,3,0,0,0.0,0.0,1.0
4,IT과학,4,0,0,0.0,0.0,1.0
5,경제,5,0,0,0.0,0.0,1.0
6,세계,6,0,0,0.0,0.0,1.0



Class Overlap. In some cases, you may want to merge classes in the top rows (below)
-----------------------------------------------------------------------------------



,Class Name A,Class Name B,Class Index A,Class Index B,Num Overlapping Examples,Joint Probability
0,생활문화,스포츠,0,1,0,0.0
1,생활문화,정치,0,2,0,0.0
2,생활문화,사회,0,3,0,0.0
3,생활문화,IT과학,0,4,0,0.0
4,생활문화,경제,0,5,0,0.0
5,생활문화,세계,0,6,0,0.0
6,스포츠,정치,1,2,0,0.0
7,스포츠,사회,1,3,0,0.0
8,스포츠,IT과학,1,4,0,0.0
9,스포츠,경제,1,5,0,0.0



 * Overall, about 0% (0 of the 1,213) labels in your dataset have potential issues.
 ** The overall label health score for this dataset is: 1.00.

Generated with <3 from Cleanlab.



In [33]:
train_df.to_csv('simon_non_noise_third_train.csv')

In [ ]:
# import numpy as np

# def analyze_top_issues(ordered_issues, pred_probs, train_df, class_names, n=1500):
#     for i, idx in enumerate(ordered_issues[:n], 1):
#         # 현재 레이블에 대한 모델의 확신도 계산
#         current_label = train_df.iloc[idx]['target']
#         confidence = pred_probs[idx][current_label]
        
#         # 모델이 가장 높은 확률로 예측한 클래스
#         predicted_class = np.argmax(pred_probs[idx])
#         predicted_confidence = pred_probs[idx][predicted_class]
        
#         print(f"\n=== {i}번째 ===")
#         print(f"텍스트: {train_df.iloc[idx]['text']}")
#         print(f"현재 레이블: {class_names[current_label]}")
#         print(f"현재 레이블에 대한 확신도: {confidence:.3f}")
#         print(f"모델이 예측한 클래스: {class_names[predicted_class]}")
#         print(f"예측 클래스에 대한 확신도: {predicted_confidence:.3f}")
        
#         # 모든 클래스에 대한 확률 출력
#         print("\n각 클래스별 확률:")
#         for class_idx, prob in enumerate(pred_probs[idx]):
#             print(f"{class_names[class_idx]}: {prob:.3f}")

# # 함수 실행
# analyze_top_issues(ordered_label_issues, pred_probs, train_df, class_names)


=== 1번째 ===
텍스트: 잉글랜드 FA컵 5부리그의 반란…서턴·링컨 16강행
현재 레이블: 경제
현재 레이블에 대한 확신도: 0.047
모델이 예측한 클래스: 스포츠
예측 클래스에 대한 확신도: 0.443

각 클래스별 확률:
생활문화: 0.180
스포츠: 0.443
정치: 0.079
사회: 0.053
IT과학: 0.116
경제: 0.047
세계: 0.082

=== 2번째 ===
텍스트: 월드컵 신태용 유임 질문에 독일 잡았지만…마음 정리...
현재 레이블: 경제
현재 레이블에 대한 확신도: 0.050
모델이 예측한 클래스: 스포츠
예측 클래스에 대한 확신도: 0.382

각 클래스별 확률:
생활문화: 0.138
스포츠: 0.382
정치: 0.121
사회: 0.058
IT과학: 0.139
경제: 0.050
세계: 0.112

=== 3번째 ===
텍스트: 황희찬 빅클럽 관심 월드컵서 좋은 모습 펼쳐야
현재 레이블: 경제
현재 레이블에 대한 확신도: 0.057
모델이 예측한 클래스: 스포츠
예측 클래스에 대한 확신도: 0.398

각 클래스별 확률:
생활문화: 0.132
스포츠: 0.398
정치: 0.105
사회: 0.051
IT과학: 0.155
경제: 0.057
세계: 0.102

=== 4번째 ===
텍스트: 아시안게임 2연패 노리던 한국 남자농구 이란에 막혀 ...
현재 레이블: 사회
현재 레이블에 대한 확신도: 0.058
모델이 예측한 클래스: 스포츠
예측 클래스에 대한 확신도: 0.480

각 클래스별 확률:
생활문화: 0.131
스포츠: 0.480
정치: 0.094
사회: 0.058
IT과학: 0.087
경제: 0.049
세계: 0.101

=== 5번째 ===
텍스트: 대한항공 우리카드 꺾고 3연승…GS칼텍스 1라운드 전승종합
현재 레이블: 경제
현재 레이블에 대한 확신도: 0.063
모델이 예측한 클래스: 스포츠
예측 클래스에 대한 확신도: 0.417

각 클래스별 확률:
생활문화: 0.149
스포츠: 0.417
정치: 0.074
사회: 